In [ ]:
"""
PICA Vergleichstool (Tkinter GUI)
==================================

Beschreibung V1-V9:
-------------
Dieses Tool vergleicht bibliografische Daten aus einer Excel-Datei mit Datensätzen im PICA-Format
(z. B. aus einem Datendump, erzeugt mit dem Tool `Pica-rs`).

Das Ziel ist es, Felder wie Titel, Signatur, Standort usw. aus der Excel-Tabelle mit den entsprechenden
Feldern im PICA-Datensatz für eine bestimmte IDN + EPN zu vergleichen.

Funktionalität:
---------------
1. Excel-Datei laden:
   - Beliebige Excel-Datei mit bibliografischen Daten (z. B. Titel, Jahr, IDN, EPN etc.)
   - Spalten werden über Dropdown-Menüs PICA-Feldern zugeordnet

2. PICA-Datendump wählen:
   - Auswahl einer .dat-Datei mit PICA-Datensätzen (UTF-8, eine Zeile pro Feld)
   - Die Datensätze werden über `pica filter` zeilenweise extrahiert

3. Vergleich starten:
   - Für jede Zeile in der Excel-Tabelle wird:
     - Die IDN gesucht
     - Die passende EPN innerhalb der IDN gefunden (Exemplarblock) + Standortblock ermittelt
     - Alle zugeordneten Felder mit den PICA-Werten verglichen
     - Ergebnis als ✅ (gleich) oder ❌ (abweichend) im Textfeld angezeigt  
        V7 nur noch abweichende Felder werden angezeigt

4. Ergebnis:
   - Übersichtliche Vergleichsausgabe für jede IDN/EPN-Kombination
   - Ideal zur Datenkontrolle, Korrekturprüfung oder Vorbereitung für Migrationsprojekte

Anwendung:
----------
1. Skript starten:         
2. "Excel-Datei laden":    Excel-Datei auswählen und Spalten zuordnen
3. "PICA-Datendump wählen": Dump-Datei auswählen (.dat, UTF-8)
4. "Vergleichen":          Vergleich starten und Ergebnisse prüfen
5. "Excel-Werte übernehmen" : Werte werden in der Exceltabelle ersetzt - Speicherabfrage

Beispiele für Feldzuordnung:
-----------------------------
  IDN            → 003@.0
  EPN            → 203@.0
  Signatur       → 209A.a
  Standort       → 209A.f
  Standortnotation → 209A.g
  AKZ            → 209C.a
  Titel          → 021A.a
  Jahr_a         → 011@.a
  Satzart        → 0500.a

Voraussetzungen:
----------------
- Python 3.x
- Abhängigkeiten: tkinter, pandas, PyYAML, unicodedata
- Das CLI-Tool `pica-rs` muss installiert und im PATH verfügbar sein!
- UTF-8-kodierter Datendump aus Aleph oder einem ähnlichen System

Hinweise:
---------
- Feldvergleiche sind normalisiert (z.B. Unicode-Normalisierung, Trimmen)
- Bei mehrfachen Vorkommen von Feldern (z.B. 209A in /XX-Blöcken) wird nach EPN selektiert
- Falsche oder fehlende Zuordnung führt zu Fehlermeldungen

Neuer Versionsinhalt:
V6 = - Innerhalb der Exemplardaten wird auch eine Standorterkennung abgefragt. Somit wird verhindert, 
       1. Exemplar /01 von Leipzig und Frankfurt gleichzeitig ausgelesen wird
V7 = - GUI zeigt nur noch gefundene Änderungen
     - Button "Excel-Werte übernehmen" damit Änderungen in die exceltabelle automatisch ersetzt werden
     - derzeit noch aktives Speichern notwendig (damit lassen sich erstmal Kopien erstellen - im Testmodus)
     - Zellen der Änderungen sind gelb hervorgehoben
     - GUI Scrollbar
     - Mausrad-Scrollstop bei Spalten-Wertzuordnung nach dem Wählen
V8 = - Mehrfach Unterfelder nun auslesebar 
     - In der Mapping Datei können nun mehrere Unterfelder mit einander kombiniert vorgegeben werden + Trennzeichen
       für die Ausgabe
V9 = - Tree statt Text ansicht 
     - Treffer in Checkbox abwählbar, damit diese nicht in die Exceltabelle übernommen werden
     - Bei eigenen Testversuchen benötigt das Programm ca. 3sek pro Datensatz - kommt auf die größe des dumps an

to-do: mehrfach belegung von Unterfeldern korrekt wiedergeben, 4000 bei Af-Sätzen bedingung schreiben geknüpft an .9

Das Mapping in der yaml-Date kann gerne individuell angepasst und erweitert werden
"""

In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
import subprocess
import unicodedata
import yaml
import os
import re
from collections import defaultdict
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

class PicaComparerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("PICA Vergleich")

        self.mapping = self.load_mapping("mapping.yaml")
        self.label_to_field = {k: v for k, v in self.mapping.items() if v}
        #self.field_to_label = {(tuple(v["fields"]), v["join"]): k for k, v in self.label_to_field.items()}

        self.df = None
        self.column_mappings = {}
        self.pica_dump_path = None  # Hier wird der Pfad zum Datendump gespeichert

        self.differences = {}  # key: DataFrame-Zeilenindex, value: dict mit Spalte → PICA-Wert
        self.setup_ui()
        

    def load_mapping(self, filepath):
        with open(filepath, "r", encoding="utf-8") as f:
            raw_mapping = yaml.safe_load(f)

        processed_mapping = {}
        for label, val in raw_mapping.items():
            if isinstance(val, str):
                # Versuch: Feldliste + Trenner erkennen, z. B. "021A.a + 021A.d : "
                if "+" in val:
                    parts = [p.strip() for p in val.split("+")]
                    # Prüfen ob letzter Teil ein Trenner ist (endet mit ": ", "; " etc.)
                    if any(parts[-1].endswith(t) for t in [":", ";", ": ", "; "]):
                        join = parts[-1][-2:].strip()
                        fields = parts[:-1]
                    else:
                        join = " "
                        fields = parts
                else:
                    fields = [val.strip()]
                    join = " "
            elif isinstance(val, dict):
                fields = val.get("fields", [])
                join = val.get("join", " ")
            elif isinstance(val, list):
                fields = val
                join = " "
            else:
                fields = [str(val)]
                join = " "

            processed_mapping[label] = {"fields": tuple(fields), "join": join}

        return processed_mapping

    def setup_ui(self):
        frame = tk.Frame(self.root)
        frame.pack(padx=10, pady=10, fill="both", expand=True)

        # Buttons oben
        tk.Button(frame, text="Excel-Datei laden", command=self.load_excel).grid(row=0, column=0, sticky="w")
        tk.Button(frame, text="PICA-Datendump wählen", command=self.select_pica_dump).grid(row=0, column=1, sticky="w", padx=10)

        # -- Scrollbarer Bereich für Spalten-Mapping --

        # Container Frame für Canvas + Scrollbars
        mapping_container = tk.Frame(frame)
        mapping_container.grid(row=1, column=0, columnspan=2, pady=10, sticky="nsew")

        # Canvas zum Scrollen
        self.mapping_canvas = tk.Canvas(mapping_container, height=200)
        self.mapping_canvas.pack(side="left", fill="both", expand=True)

        # Vertikale Scrollbar
        v_scrollbar = tk.Scrollbar(mapping_container, orient="vertical", command=self.mapping_canvas.yview,
                           bg="blue", activebackground="blue", troughcolor="blue", width=20)
        v_scrollbar.pack(side="right", fill="y")

        # Horizontale Scrollbar
        h_scrollbar = tk.Scrollbar(frame, orient="horizontal", command=self.mapping_canvas.xview, width=20)
        h_scrollbar.grid(row=2, column=0, columnspan=2, sticky="ew")
        h_scrollbar.config(bg="lightblue", activebackground="blue")

        self.mapping_canvas.configure(yscrollcommand=v_scrollbar.set, xscrollcommand=h_scrollbar.set)

        # Innerer Frame im Canvas, in dem die Mapping-Widgets leben
        self.mapping_frame = tk.Frame(self.mapping_canvas)
        self.mapping_canvas.create_window((0, 0), window=self.mapping_frame, anchor="nw")

        # Damit der Canvas die Größe des inneren Frames kennt und scrollbar macht:
        def on_mapping_frame_configure(event):
            self.mapping_canvas.configure(scrollregion=self.mapping_canvas.bbox("all"))

        self.mapping_frame.bind("<Configure>", on_mapping_frame_configure)

        # Button Vergleich starten
        tk.Button(frame, text="Vergleichen", command=self.compare).grid(row=3, column=0, columnspan=2, pady=5)

        
        # Textfeld Ergebnis + Scrollbar wie vorher
        result_frame = tk.Frame(frame)
        result_frame.grid(row=4, column=0, columnspan=2, pady=10, sticky="nsew")

        self.result_tree = ttk.Treeview(
            result_frame, 
            columns=("Auswahl", "IDN", "EPN", "Feld", "Excel", "PICA"),
            show="headings", 
            selectmode="none"  # Wir steuern Auswahl selbst per Klick
        )
        self.result_tree.pack(side="left", fill="both", expand=True)

        # Spaltenüberschriften
        self.result_tree.heading("Auswahl", text="Auswählen")
        self.result_tree.column("Auswahl", width=60, anchor="center")

        for col in ("IDN", "EPN", "Feld", "Excel", "PICA"):
            self.result_tree.heading(col, text=col)
            self.result_tree.column(col, width=150)

        # Scrollbar bleibt unverändert
        scrollbar = tk.Scrollbar(result_frame, command=self.result_tree.yview)
        scrollbar.pack(side="right", fill="y")
        self.result_tree.configure(yscrollcommand=scrollbar.set)

        # Event zum Klick auf Spalte Auswahl
        self.result_tree.bind("<Button-1>", self.on_tree_click)

        # Dict zum Speichern Auswahl-Status der Items (Item-ID -> bool)
        self.checked_items = {}

        # Label für PICA-Datendump-Pfad
        self.pica_path_label = tk.Label(frame, text="Kein PICA-Datendump ausgewählt")
        self.pica_path_label.grid(row=5, column=0, columnspan=2, sticky="w")

        # Frame Konfiguration für dynamische Größenanpassung
        frame.grid_rowconfigure(4, weight=1)  # Result Text wächst vertikal
        frame.grid_columnconfigure(0, weight=1)
        
        self.update_button = tk.Button(frame, text="Excel-Werte übernehmen", command=self.update_excel_values)
        self.update_button.grid(row=6, column=0, columnspan=2, pady=5)
        self.update_button.grid_remove()  # Button verstecken bis Vergleich läuft
        
    def on_tree_click(self, event):
        region = self.result_tree.identify("region", event.x, event.y)
        if region != "cell":
            return

        col = self.result_tree.identify_column(event.x)
        if col != "#1":  # nur erste Spalte (Auswahl) bearbeiten
            return

        row_id = self.result_tree.identify_row(event.y)
        if not row_id:
            return

        current = self.checked_items.get(row_id, True)  # default ausgewählt = True
        self.checked_items[row_id] = not current

        # Text in Auswahl-Spalte aktualisieren
        display = "✓" if self.checked_items[row_id] else ""
        self.result_tree.set(row_id, "Auswahl", display)

    def load_excel(self):
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if not file_path:
            return

        self.df = pd.read_excel(file_path, dtype=str).fillna("")

        # Mapping Frame neu befüllen
        for widget in self.mapping_frame.winfo_children():
            widget.destroy()

        self.column_mappings.clear()

        def disable_mousewheel(event):
            return "break"  # verhindert Standardverhalten

        for i, column in enumerate(self.df.columns):
            tk.Label(self.mapping_frame, text=column).grid(row=i, column=0, sticky="w")
            combo = ttk.Combobox(self.mapping_frame, values=["nichts zuordnen"] + list(self.label_to_field.keys()))
            combo.set(column if column in self.label_to_field else "nichts zuordnen")
            combo.grid(row=i, column=1)

            combo.bind("<MouseWheel>", disable_mousewheel)  # Windows
            combo.bind("<Button-4>", disable_mousewheel)    # Linux scroll up
            combo.bind("<Button-5>", disable_mousewheel)    # Linux scroll down

            self.column_mappings[column] = combo

    def select_pica_dump(self):
        file_path = filedialog.askopenfilename(title="PICA-Datendump auswählen", filetypes=[("Datendump files", "*.dat"), ("Alle Dateien", "*.*")])
        if file_path:
            self.pica_dump_path = file_path
            self.pica_path_label.config(text=f"PICA-Datendump: {file_path}")

    
    def split_into_exemplar_blocks(self, lines):
        blocks = {}
        for line in lines:
            if "/" not in line:
                continue
            try:
                field, rest = line.split(" ", 1)
                if "/" in field:
                    base, block = field.split("/")
                    blocks.setdefault(block, []).append((base, rest.strip()))
            except ValueError:
                continue
        return blocks

    def find_exemplar_block_by_epn(self, blocks, target_epn):
        for block_id, entries in blocks.items():
            for base, content in entries:
                if base == "203@":
                    subfields = content.split("")
                    for sub in subfields:
                        if sub.startswith("0") and sub[1:] == target_epn:
                            return entries
        return None

    def run_pica_query(self, idn):
        if not self.pica_dump_path:
            messagebox.showerror("Fehler", "Bitte zuerst einen PICA-Datendump auswählen.")
            return []

        cmd = f'pica filter -s "003@.0 == \\"{idn}\\"" "{self.pica_dump_path}"'
        print(f"🔍 Starte PICA-Abfrage: {cmd}")
        try:
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True, encoding="utf-8")
            lines = result.stdout.splitlines()


            return lines
        except Exception as e:
            print(f"[Fehler bei Abfrage von IDN {idn}: {e}]")
            return []

    def normalize_str(self, s):
        if s is None:
            return ""
        s = unicodedata.normalize("NFC", str(s).strip())
        s = re.sub(r"\s+", " ", s)  # ersetzt mehrere Leerzeichen durch eines
        return s
    
    def parse_stammdaten(self, lines):
        stammdaten = {}
        for line in lines:
            if "/" not in line:
                try:
                    field, rest = line.split(" ", 1)
                    stammdaten[field] = rest.strip()
                except ValueError:
                    continue
        return stammdaten
    
    def parse_location_blocks(self, lines):
        location_blocks = defaultdict(lambda: defaultdict(list))
        current_location = None
        for line in lines:
            if line.startswith("101@"):
                match = re.search(r"a(\d+)", line)
                if match:
                    current_location = match.group(1)
            elif "/" in line and current_location:
                try:
                    field, rest = line.split(" ", 1)
                    if "/" in field:
                        base, suffix = field.split("/")
                        location_blocks[current_location][suffix].append((base, rest.strip()))
                except ValueError:
                    continue
        return location_blocks

    
    def find_block_by_epn_location(self, location_blocks, epn):
        for location, blocks in location_blocks.items():
            for suffix, entries in blocks.items():
                for base, content in entries:
                    if base == "203@" and any(sub.startswith("0") and sub[1:] == epn for sub in content.split("")):
                        return entries
        return None
    
    def parse_exemplar_blocks(self, lines):
        blocks = defaultdict(dict)
        for line in lines:
            match = re.match(r"^(\d{3}[A-Z])(?:/(\d{2}))?\s(.+)", line)
            if match:
                field, suffix, content = match.groups()
                suffix = suffix or "00"  # kein /XX → allgemeiner Block
                blocks[suffix][field] = content
        return blocks
    
    
    def extract_fields_from_block(self, block_entries, pica_keys, joiner=" "):
        values = []
        for pica_key in pica_keys:
            if '.' not in pica_key:
                continue
            field, sub = pica_key.split('.', 1)
            for f, line in block_entries:
                if f != field:
                    continue
                values.extend([part[1:] for part in line.split("") if part.startswith(sub)])
        return joiner.join(values)
    
    def compare(self):
        if self.df is None:
            messagebox.showerror("Fehler", "Keine Excel-Datei geladen.")
            return

        if not self.pica_dump_path:
            messagebox.showerror("Fehler", "Bitte zuerst einen PICA-Datendump auswählen.")
            return

        self.result_tree.delete(*self.result_tree.get_children())  # Vor der Schleife löschen
        self.differences.clear()  # Alte Unterschiede löschen

        idn_column = self.get_column_by_mapping("IDN")
        if not idn_column:
            messagebox.showerror("Fehler", "Keine Spalte für IDN zugeordnet.")
            return

        for idx, row in self.df.iterrows():
            idn = row[idn_column]
            if not idn:
                continue

            pica_lines = self.run_pica_query(idn)
            full_record = [(line.split(" ", 1)[0], line.split(" ", 1)[1]) for line in pica_lines if " " in line]
            location_blocks = self.parse_location_blocks(pica_lines)

            epn_column = self.get_column_by_mapping("EPN")
            target_epn = self.normalize_str(row[epn_column]) if epn_column else ""
            exemplar_block = self.find_block_by_epn_location(location_blocks, target_epn)

            if not exemplar_block:
                continue

            row_diffs = {}

            for col, combo in self.column_mappings.items():
                label = combo.get()
                if label == "nichts zuordnen" or label not in self.label_to_field:
                    continue
                if label == "EPN":
                    continue

                mapping_entry = self.mapping[label]
                pica_keys = mapping_entry["fields"]
                joiner = mapping_entry.get("join", " ")
                excel_val = row[col]
                excel_val_norm = self.normalize_str(excel_val)

                if pica_keys[0].split(".")[0] in {"209A", "208@", "203@", "209B", "209C", "245Y", "245Z"}:
                    pica_val = self.extract_fields_from_block(exemplar_block, pica_keys, joiner)
                else:
                    pica_val = self.extract_fields_from_block(full_record, pica_keys, joiner)

                pica_val_norm = self.normalize_str(pica_val)

                if excel_val_norm != pica_val_norm:
                    row_diffs[col] = pica_val  # Spalte und neuer Wert merken

            if row_diffs:
                self.differences[idx] = row_diffs
                for col, pica_val in row_diffs.items():
                    label = self.column_mappings[col].get()
                    excel_val = self.df.loc[idx, col]
                    epn_val = self.df.loc[idx, epn_column] if epn_column else ""
                    idn_val = self.df.loc[idx, idn_column]

                    row_id = self.result_tree.insert(
                        "", "end",
                        values=("✓", idn_val, epn_val, label, excel_val, pica_val)
                    )
                    self.checked_items[row_id] = True       
        
        if self.differences:
            self.update_button.grid()  # Button anzeigen
        else:
            # Kein result_text mehr vorhanden -> stattdessen evtl. ein MessageBox:
            messagebox.showinfo("Info", "Keine Unterschiede gefunden.")
            self.update_button.grid_remove()


    def get_column_by_mapping(self, label):
        for col, combo in self.column_mappings.items():
            if combo.get() == label:
                return col
        return None
    
    def get_excel_column_by_label(self, label):
        # Geht durch alle Spalten mit Combo-Boxen und sucht, welche Combo den label-Text hat
        for col, combo in self.column_mappings.items():
            if combo.get() == label:
                return col
        return None
    
    def update_excel_values(self):
        if not self.differences:
            messagebox.showinfo("Info", "Keine Unterschiede zum Übernehmen.")
            return

        idn_column = self.get_column_by_mapping("IDN")
        if not idn_column:
            messagebox.showerror("Fehler", "Keine IDN-Spalte zugeordnet.")
            return

        for item_id in self.result_tree.get_children():
            if self.checked_items.get(item_id, True):
                values = self.result_tree.item(item_id, "values")
                idn = values[1]
                feld = values[3]
                pica_wert = values[5]

                feld = values[3]  # das Label, z.B. "Titel"
                excel_col = self.get_excel_column_by_label(feld)  # tatsächlicher DataFrame-Spaltenname
                if excel_col is None:
                    continue  # oder Fehlermeldung

                # DataFrame-Zeilenindex anhand von idn finden
                matches = self.df.index[self.df[idn_column] == idn].tolist()
                if matches:
                    idx = matches[0]
                    self.df.at[idx, excel_col] = pica_wert
                    # self.differences aktualisieren (optional)
                    if idx in self.differences:
                        self.differences[idx][excel_col] = pica_wert

        save_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel Dateien", "*.xlsx")])
        if save_path:
            self.df.to_excel(save_path, index=False)
            self.highlight_changes_in_excel(save_path)
            messagebox.showinfo("Erfolg", f"Excel-Datei wurde gespeichert und Änderungen farbig markiert:\n{save_path}")

        self.update_button.grid_remove()
        
    def highlight_changes_in_excel(self, filepath):
        wb = load_workbook(filepath)
        ws = wb.active

        # Farbe für geänderte Zellen (hellgelb)
        fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")

        # Mapping: DataFrame-Spalten zu Excel-Spaltenbuchstaben ermitteln
        col_idx_to_excel_col = {}
        for i, col_name in enumerate(self.df.columns):
            col_letter = ws.cell(row=1, column=i+1).column_letter
            col_idx_to_excel_col[col_name] = col_letter

        for row_idx, changes in self.differences.items():
            excel_row = row_idx + 2  # +2 wegen Header (Excel beginnt bei 1)
            for col_name in changes.keys():
                col_letter = col_idx_to_excel_col.get(col_name)
                if col_letter:
                    cell = ws[f"{col_letter}{excel_row}"]
                    cell.fill = fill

        wb.save(filepath)

if __name__ == "__main__":
    root = tk.Tk()
    app = PicaComparerApp(root)
    root.mainloop()
